# update7 - TBD

In [1]:
import sys

In [2]:
sys.path.append('../..')

In [3]:
from datetime import datetime, timezone
import json
from tqdm import tqdm
from pathlib import Path
import gzip
import pickle

In [4]:
import pandas as pd
import numpy as np

In [5]:
import rapidfuzz

In [6]:
from app import db
from models import Source, ISSNtoISSNL
from models.source import DELETED_SOURCE_ID
from sqlalchemy import text
from sqlalchemy.orm import Load
from sqlalchemy.exc import MultipleResultsFound

140200128323968: loading merged_into_institutions_dict
140200128323968: loading valid concept IDs
140200128323968: loading valid topic IDs
140200128323968: loading valid keyword IDs


In [7]:
from cleanup.util import make_request, paginate_openalex


In [8]:
%%time
sq = """select if.*, ife.cluster_title from issn_audit_20240321.issn_ic_datafile_202402 if
left join issn_audit_20240321.issn_ic_datafile_expanded_202402 ife
  on if."submitted_1348-0278"  = ife.issns;"""
df_issnl_file = pd.read_sql_query(sq, db.engine)

CPU times: user 679 ms, sys: 91.9 ms, total: 771 ms
Wall time: 1.78 s


In [9]:
df_issnl_file = df_issnl_file.rename(columns={"submitted_1348-0278": "submitted_issn"})

In [10]:
%%time
sq = """select journal_id, display_name, issn, issns, issns_text_array, type, country_code, alternate_titles, publisher_id
    from mid.journal
    where merge_into_id is null
    and issn is not null"""
df_midjournal = pd.read_sql_query(sq, db.engine)

CPU times: user 1.02 s, sys: 42 ms, total: 1.06 s
Wall time: 2.47 s


In [11]:
print(len(df_midjournal))
df_midjournal.dropna(subset='issn', inplace=True)
print(len(df_midjournal))

139932
139932


Are there duplicate ISSN-Ls in `mid.journal`?

In [12]:
num_duplicates = df_midjournal['issn'].duplicated().sum()
print(f"There are {num_duplicates} duplicates")

There are 0 duplicates


:party:

Does every ISSN-L in `mid.journal` have the same ISSN as the first item in the ISSN list?

In [13]:
%%time

def _is_first_in_issns(row):
    issnl = row['issn']
    issn_list = json.loads(row['issns'])
    return issnl == issn_list[0]

def _is_first_in_issns_text_array(row):
    issnl = row['issn']
    issn_list = row['issns_text_array']
    return issnl == issn_list[0]

df_midjournal['issnl_is_first_in_issns'] = df_midjournal.apply(_is_first_in_issns, axis=1)
df_midjournal['issnl_is_first_in_issns_text_array'] = df_midjournal.apply(_is_first_in_issns_text_array, axis=1)

CPU times: user 1.32 s, sys: 0 ns, total: 1.32 s
Wall time: 1.32 s


In [14]:
df_midjournal['issnl_is_first_in_issns'].value_counts()

issnl_is_first_in_issns
True    139932
Name: count, dtype: int64

In [15]:
df_midjournal['issnl_is_first_in_issns_text_array'].value_counts()

issnl_is_first_in_issns_text_array
True    139932
Name: count, dtype: int64

:party:

In [16]:
df_issn_to_issnl = pd.read_sql_query("""select * from mid.journal_issn_to_issnl""", db.engine)

In [17]:
issn_to_issnl = df_issn_to_issnl.set_index('issn', verify_integrity=True)['issnl']

In [18]:
issnls = issn_to_issnl.unique()

In [19]:
# get dict of issnl to issn
x = df_issn_to_issnl.dropna(subset=['issnl'])
# x = x[x['category'].isin(['Register', 'Work', 'Free', 'Validation Request'])]
x = x[['issn', 'issnl']]
issnl_to_issn = {}
ignore = [
    '0000-0000',
]
x = x[~(x['issnl'].isin(ignore))]
for name, gbdf in x.groupby('issnl'):
    if name in ignore:
        continue
    issnl_to_issn[name] = gbdf['issn'].tolist()
len(issnl_to_issn)

141008

Does every ISSN resolve to at most one Source?

In [20]:
smap = {}
for source_id, issn_list in tqdm(df_midjournal.set_index('journal_id', verify_integrity=True)['issns_text_array'].items(), total=len(df_midjournal)):
    for issn in issn_list:
        if issn in smap:
            smap[issn].append(source_id)
        else:
            smap[issn] = [source_id]

  0%|          | 0/139932 [00:00<?, ?it/s]

100%|██████████| 139932/139932 [00:00<00:00, 368644.91it/s]


In [21]:
d = []
for issn, source_list in smap.items():
    d.append({
        'issn': issn,
        'num_sources': len(source_list),
    })
_df = pd.DataFrame(d)
_df['num_sources'].value_counts()
df_issnl_file['num_sources_resolve'] = df_issnl_file['submitted_issn'].map(_df.set_index('issn', verify_integrity=True)['num_sources'])
df_issnl_file['num_sources_resolve'].fillna(value=0, inplace=True)

In [22]:
%%time
sq = """select journal_id, display_name, issn, issns, issns_text_array, type, merge_into_id, merge_into_date
    from mid.journal
    ;
    """
df_midjournal_withmerged = pd.read_sql_query(sq, db.engine)

CPU times: user 1.09 s, sys: 108 ms, total: 1.2 s
Wall time: 1.67 s


In [23]:
smap_withmerged = {}
for source_id, issn_list in tqdm(df_midjournal_withmerged.set_index('journal_id', verify_integrity=True)['issns_text_array'].items(), total=len(df_midjournal_withmerged)):
    if issn_list:
        for issn in issn_list:
            if issn in smap_withmerged:
                smap_withmerged[issn].append(source_id)
            else:
                smap_withmerged[issn] = [source_id]

  0%|          | 0/262745 [00:00<?, ?it/s]

100%|██████████| 262745/262745 [00:00<00:00, 508622.75it/s] 


In [24]:
d = []
for issn, source_list in smap_withmerged.items():
    d.append({
        'issn': issn,
        'num_sources': len(source_list),
    })
_df = pd.DataFrame(d)
_df['num_sources'].value_counts()
df_issnl_file['num_sources_resolve_withmerged'] = df_issnl_file['submitted_issn'].map(_df.set_index('issn', verify_integrity=True)['num_sources'])
df_issnl_file['num_sources_resolve_withmerged'].fillna(value=0, inplace=True)

In [25]:
df_issnl_file['num_sources_resolve'].value_counts()

num_sources_resolve
1.0    211910
0.0      2696
Name: count, dtype: int64

In [26]:
df_issnl_file['num_sources_resolve_withmerged'].value_counts()

num_sources_resolve_withmerged
1.0    203810
2.0      8798
0.0      1776
3.0       203
5.0        12
4.0         6
6.0         1
Name: count, dtype: int64

In [28]:
# start with submitted_issns that are ISSNLs
df_to_add = df_issnl_file[df_issnl_file['num_sources_resolve']==0]
df_to_add = df_to_add[(df_to_add['submitted_issn'] == df_to_add['issnl'])]
print(len(df_to_add))

1242


In [29]:
(df_to_add['submitted_issn'] == df_to_add['issn']).value_counts()

True    1242
Name: count, dtype: int64

In [30]:
(df_to_add['submitted_issn'] == df_to_add['issnl']).value_counts()

True    1242
Name: count, dtype: int64

In [31]:
df_to_add['num_sources_resolve_withmerged'].value_counts()

num_sources_resolve_withmerged
0.0    1163
1.0      67
2.0       6
5.0       5
3.0       1
Name: count, dtype: int64

In [32]:
d = []
midjournal_lookup = df_midjournal_withmerged.set_index('journal_id', verify_integrity=True)
for submitted_issn, row in df_to_add[df_to_add['num_sources_resolve_withmerged']>=1].set_index('submitted_issn', verify_integrity=True).iterrows():
    this_source_ids = smap_withmerged[submitted_issn]
    for source_id in this_source_ids:
        midjournal_row = midjournal_lookup.loc[source_id]
        d.append({
            'submitted_issn': submitted_issn,
            'source_id': source_id,
            'cluster_title': row['cluster_title'],
            'display_name': midjournal_row['display_name'],
            'midjournal_issn': midjournal_row['issn'],
            'midjournal_issns': midjournal_row['issns'],
            'midjournal_issns_text_array': midjournal_row['issns_text_array'],
            'midjournal_type': midjournal_row['type'],
            'merge_into_id': midjournal_row['merge_into_id'],
            'merge_into_date': midjournal_row['merge_into_date'],
            'num_sources_resolve_withmerged': row['num_sources_resolve_withmerged'],
        })
df_unmerge = pd.DataFrame(d)
df_unmerge['deleted'] = df_unmerge['merge_into_id'] == DELETED_SOURCE_ID


In [33]:
df_to_add

,index,submitted_issn,issn,issnl,category,medium,frequency,country,center,language,start,end,title,comment,resolved,cluster_title,num_sources_resolve,num_sources_resolve_withmerged
0,165211,2221-9781,2221-9781,2221-9781,Register,Online,Monthly,UKRAINE,Ukraine,Multiple languages,2009,9999,Lìki Ukraïni plûs,,False,Lìki Ukraïni plûs,0.0,0.0
1,185888,2546-2202,2546-2202,2546-2202,Register,Online,Annual,TANZANIA (UNITED REPUBLIC OF),"Tanzania, United Republic of",Swahili,2017,9999,Mulika (Online),,False,Mulika,0.0,0.0
2,171585,2720-9512,2720-9512,2720-9512,Register,Online,Three times a year,INDONESIA,Indonesia,Multiple languages,2020,9999,PELAGICUS (Online),,False,PELAGICUS,0.0,0.0
4,185113,2828-5263,2828-5263,2828-5263,Register,Online,Semiannual,INDONESIA,Indonesia,Indonesian,2022,9999,Afore (Online),,False,Afore,0.0,0.0
7,182967,2992-7358,2992-7358,2992-7358,Register,Online,Semiannual,MEXICO,Mexico,Spanish,2021,9999,Didac (México),,False,Didac,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196128,118998,2692-7691,2692-7691,2692-7691,Register,Online,Monthly,UNITED STATES,United States,English,2022,9999,American journal of biological anthropology,,True,American journal of biological anthropology,0.0,1.0
196723,171448,2828-9331,2828-9331,2828-9331,Register,Online,Semiannual,INDONESIA,Indonesia,Indonesian,2022,9999,Ushuly (Online),,False,Ushuly,0.0,0.0
196966,163130,2964-6286,2964-6286,2964-6286,Register,Online,Quarterly,INDONESIA,Indonesia,Indonesian,2023,9999,AlFihris (Online),,False,AlFihris,0.0,0.0
196988,166617,2965-1204,2965-1204,2965-1204,Register,Online,Annual,BRAZIL,Brazil,Portuguese,2022,9999,Revista Cidadania em Foco,,False,Revista Cidadania em Foco,0.0,0.0


In [34]:
def add_new_source(issnl, issn_list, title, source_type='journal', updated_date=None, session=None):
    if session is None:
        session = db.session
    if updated_date is None:
        updated_date = datetime.now(timezone.utc).isoformat()

    # make updates
    new_issns = json.dumps(issn_list)
    new_display_name = title
    new_type = source_type
    source = Source(
        display_name = new_display_name,
        issn = issnl,
        issns = new_issns,
        issns_text_array = issn_list,
        type = new_type,
        created_date = updated_date,
        updated_date = updated_date,
    )
    session.add(source)
    session.commit()
    session.refresh(source)
    source_id = int(source.id)

    # insert into log table
    sq = """INSERT INTO issn_audit_20240321.update7
        (issnl, new_issns, new_display_name, updated_date, source_id, new_type, is_new_source, created_date)
        VALUES(:issnl, :new_issns, :new_display_name, :updated_date, :source_id, :new_type, true, :updated_date);
        """
    session.execute(text(sq), {
        'issnl': issnl,
        'new_issns': new_issns,
        'new_display_name': new_display_name,
        'updated_date': updated_date,
        'source_id': source_id,
        'new_type': new_type,
    }) 
    return source_id

In [45]:
try:
    x = db.session.execute("""select issnl from issn_audit_20240321.update7""").scalars().all()
finally:
    db.session.close()

In [47]:
print(len(x))
print(len(set(x)))

617
617


In [53]:
# case where submitted_issn == issnl
# Add a new source. Merge any existing (already merged) sources into this new one.
try:
    seen_issnls = db.session.execute("""select issnl from issn_audit_20240321.update7""").scalars().all()
    seen_issnls = set(seen_issnls)
    for issnl, row in tqdm(df_to_add.set_index('issnl', verify_integrity=True).iterrows(), total=len(df_to_add)):
        if issnl in seen_issnls:
            continue
        updated_date = datetime.now(timezone.utc).isoformat()
        issn_list = issnl_to_issn[issnl]
        if issnl not in issn_list:
            issn_list.append(issnl)
        # move issnl to the front
        issn_list.insert(0, issn_list.pop(issn_list.index(issnl)))
        title = row['cluster_title']
        if not title:
            title = ''
        if 'conference' in title.lower():
            source_type = 'conference'
        elif 'book series' in title.lower():
            source_type = 'book series'
        else:
            source_type = 'journal'
        source_id = add_new_source(issnl, issn_list, title=title, source_type=source_type, updated_date=updated_date, session=db.session)
        for _, row in df_unmerge[df_unmerge['midjournal_issn']==issnl].iterrows():
            if row['deleted'] == False:
                # merge into new source
                source_id = int(row['source_id'])
                source = db.session.query(Source).filter_by(journal_id=source_id).one()
                old_merge_into_id = row['merge_into_id']
                new_merge_into_id = source_id
                old_merge_into_date = row['merge_into_date']
                if pd.isna(old_merge_into_date):
                    old_merge_into_date = None
                new_merge_into_date = updated_date
                source.merge_into_id = new_merge_into_id
                source.merge_into_date = new_merge_into_date
                source.updated_date = updated_date
                db.session.add(source)

                # insert into log table
                sq = """INSERT INTO issn_audit_20240321.update7
                    (issnl, updated_date, source_id, old_merge_into_id, merge_into_id, old_merge_into_date, merge_into_date, is_new_source)
                    VALUES(:issnl, :updated_date, :source_id, :old_merge_into_id, :merge_into_id, :old_merge_into_date, :merge_into_date, false);
                    """
                db.session.execute(text(sq), {
                    'issnl': issnl,
                    'updated_date': updated_date,
                    'source_id': source_id,
                    'old_merge_into_id': old_merge_into_id,
                    'merge_into_id': new_merge_into_id,
                    'old_merge_into_date': old_merge_into_date,
                    'merge_into_date': new_merge_into_date,
                }) 

                sq = """update issn_audit_20240321.issn_ic_datafile_202402 set resolved = true
                    where "submitted_1348-0278" = :issn"""
                db.session.execute(text(sq), {
                    'issn': issnl,
                })
        # commit for each new source
        db.session.commit()
finally:
    db.session.close()

100%|██████████| 1242/1242 [00:41<00:00, 29.96it/s] 
